<center><img src='https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSqsQO7fHV7LPSk0XzgFhXavflTpp9dumLmjQ&usqp=CAU' width = '800px'></center>

<center><img src='https://i.imgur.com/SZQXq1q.png' width = '600px'></center>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso,Ridge
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import warnings
warnings.filterwarnings('ignore')

<h1 id="heading1">
<span style="font-size:35px;color:#66FF00;font-weight:bold"><center>Train Data</center></span>
<a class="anchor-link" href="https://www.kaggle.com/alampalsingh/notebook004d896e8c#heading1" target ='_self'></a>
</h1>

<h1 id="heading5">
<span style="font-size:25px;color:#800080"><center>Loading Data</center></span>
<a class="anchor-link" href="https://www.kaggle.com/alampalsingh/notebook004d896e8c#heading5" target ='_self'></a>
</h1>

In [ ]:
df = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

In [ ]:
df

<h1 id="heading2">
<span style="font-size:25px;color:#800080"><center>Checking for NaN value.</center></span>
<a class="anchor-link" href="https://www.kaggle.com/alampalsingh/notebook004d896e8c#heading2" target ='_self'></a>
</h1>
<span style="font-size:15px;"><center>*We will drop the column if nan values are more than 30%*</center></span>


In [ ]:
null = {}
drop = []
for i in range(0,df.shape[1]):
    if 0 < df.iloc[:,i].isna().sum() <= df.shape[0]*(0.3):
        null[df.columns[i]] = df.iloc[:,i].isna().sum()
    elif df.iloc[:,i].isna().sum() > df.shape[0]*(0.3):# If the null percentage is greater than 30% we will drop that columns. 
        drop.append(df.columns[i])

In [ ]:
null,drop

In [ ]:
nullk = list(null.keys())
nullv = [float(null[k]) for k in nullk]
plt.figure(figsize=(28,12))
gr = sns.barplot(nullk,nullv,palette="coolwarm_r" )
gr.text(9,200,str('Name of column \n  vs Nan values'),fontdict = dict(color='#66FF00',fontsize = 40),weight = 'bold')

In [ ]:
df.drop(drop,axis = 1,inplace = True)

<h1 id="heading3">
<span style="font-size:25px;color:#800080"><center>Filling NaN value.</center></span>
<a class="anchor-link" href="https://www.kaggle.com/alampalsingh/notebook004d896e8c#heading3" target ='_self'></a>
</h1>

In [ ]:
df.LotFrontage.plot(kind = 'density')
plt.show()#For checking the distribution of the curve.

In [ ]:
df.describe()

In [ ]:
df.LotFrontage.fillna(df.LotFrontage.median(),inplace= True)

In [ ]:
df.MasVnrType.describe()

In [ ]:
df.MasVnrType.fillna('None',inplace = True )
df.MasVnrArea.fillna(0.0,inplace = True)

In [ ]:
for i in (29,30,31,32,34):
    df.iloc[:,i].fillna(df.iloc[:,i].describe().top,inplace=True)#Filling with most common or the top value.

In [ ]:
sns.countplot(df.Electrical,palette="coolwarm_r")


In [ ]:
df.Electrical.fillna('SBrkr',inplace=True)

In [ ]:
_,ax = plt.subplots(1,4,figsize=(30,10))
sns.countplot(df.GarageType,ax = ax[0],palette="coolwarm_r")
sns.countplot(df.GarageFinish,ax = ax[1],palette="coolwarm_r")
sns.countplot(df.GarageQual,ax = ax[2],palette="coolwarm_r")
sns.countplot(df.GarageCond,ax = ax[3],palette="coolwarm_r")

In [ ]:
df.columns.get_loc('GarageType')

In [ ]:
for i in (56,57,58,61,62):
    print(df.iloc[:,i].dtypes)

In [ ]:
df.iloc[:,56].isna().sum()

In [ ]:
df.GarageType.fillna('Attchd',inplace=True)

In [ ]:
df.GarageYrBlt.fillna(df.groupby('GarageType')['GarageYrBlt'].transform('median'),inplace=True)

In [ ]:
df.GarageFinish.fillna('UnF',inplace=True)
df.GarageQual.fillna('TA',inplace=True)
df.GarageCond.fillna('TA',inplace=True)

In [ ]:
null = {}
for i in range(0,df.shape[1]):
    if 0 < df.iloc[:,i].isna().sum():
        null[df.columns[i]] = df.iloc[:,i].isna().sum()
null
#Checking if any nan value is still remaining

<h1 id="heading4">
<span style="font-size:25px;color:#800080"><center>Dummies for categorical columns.</center></span>
<a class="anchor-link" href="https://www.kaggle.com/alampalsingh/notebook004d896e8c#heading4" target ='_self'></a>
</h1>

In [ ]:
df = pd.get_dummies(df)

In [ ]:
df

<h1 id="heading6">
<span style="font-size:35px;color:#66FF00;font-weight:bold"><center>Test Data</center></span>
<a class="anchor-link" href="https://www.kaggle.com/alampalsingh/notebook004d896e8c#heading6" target ='_self'></a>
</h1>

In [ ]:
null = []
drop = []
for i in range(0,test.shape[1]):
    if 0 < test.iloc[:,i].isna().sum() <= test.shape[0]*(0.3):
        null.append(test.columns[i]) 
    elif test.iloc[:,i].isna().sum() > test.shape[0]*(0.3):
        drop.append(test.columns[i])

In [ ]:
drop

In [ ]:
test.drop(drop,axis = 1,inplace = True)

In [ ]:
test.LotFrontage.plot(kind = 'density')
plt.show()

In [ ]:
test.LotFrontage.fillna(test.LotFrontage.median(),inplace= True)

In [ ]:
test.MasVnrType.fillna('None',inplace = True )
test.MasVnrArea.fillna(0.0,inplace = True)

In [ ]:
for i in range(0,len(null)):
    if test[null[i]].dtype == object:
        test[null[i]].fillna(test[null[i]].describe().top,inplace = True)
    else:
        test[null[i]].fillna(df.groupby('MSSubClass')[null[i]].transform('median'),inplace = True)
    

In [ ]:
test = pd.get_dummies(test)

In [ ]:
test

In [ ]:
a = []
for i in list(test):
    if i not in list(df):
        a.append(i)

In [ ]:
iD = test.Id

In [ ]:
test.drop(a,axis =1,inplace = True)

In [ ]:
test.drop(['Id'],axis =1,inplace=True)

In [ ]:
test

In [ ]:
b = []
for i in list(df):
    if i not in list(test):
        b.append(i)

In [ ]:
x = df.drop(b,axis =1)

In [ ]:
y = (df.SalePrice)

<h1 id="heading7">
<span style="font-size:30px;color:#800080"><center>df.SalePrice vs log(df.SalePrice)</center></span>
<a class="anchor-link" href="https://www.kaggle.com/alampalsingh/notebook004d896e8c#heading7" target ='_self'></a>
</h1>

In [ ]:
_,ax = plt.subplots(1,2,figsize = (20,5))
a = sns.kdeplot((y),ax = ax[0])
b = sns.kdeplot(np.log(y),ax = ax[1])
b.text(13,0.9,str('With log'),fontdict = dict(color = '#02cdfb',fontsize = 20))

<span>Log(df.SalePrice) is very well distributed having very low skewness whereas df.SalePrice is right skewed.</span>

In [ ]:
y = np.log(df.SalePrice)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state =42)

In [ ]:
rmse_score = {}

In [ ]:
las = Lasso(alpha=0.001)
las.fit(x_train,y_train)
rmse = np.sqrt(mean_squared_error((y_test),(las.predict(x_test))))
rmse_score['Lasso'] = rmse
rmse


In [ ]:
rid = Ridge(alpha=50)
rid.fit(x_train,y_train)
rmse = np.sqrt(mean_squared_error((y_test),(rid.predict(x_test))))
rmse_score['Ridge'] = rmse
rmse


In [ ]:
rfr = RandomForestRegressor(random_state=42)
rfr.fit(x_train,y_train)
rmse = np.sqrt(mean_squared_error((y_test),(rfr.predict(x_test))))
rmse_score['Random Forest'] = rmse
rmse


In [ ]:
xgb = XGBRegressor(learning_rate = 0.1)
xgb.fit(x_train,y_train)
rmse = np.sqrt(mean_squared_error((y_test),(xgb.predict(x_test))))
rmse_score['XGBoost'] = rmse
rmse



In [ ]:
lgb = LGBMRegressor(learning_rate=0.1)
lgb.fit(x_train,y_train)
rmse = np.sqrt(mean_squared_error((y_test),(lgb.predict(x_test))))
rmse_score['LightGBM'] = rmse
rmse


In [ ]:
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor

In [ ]:
cat = CatBoostRegressor(learning_rate=0.01,iterations=4000)
cat.fit(x_train,y_train,verbose = 0)
rmse = np.sqrt(mean_squared_error((y_test),(cat.predict(x_test))))
rmse_score['CatBoost'] = rmse
rmse

In [ ]:
def cross_val(X, y, model,param, folds=5):

    skf = KFold(n_splits=folds, shuffle=True, random_state=42)
    for fold, (train_idx, test_idx) in enumerate(skf.split(X, y)):
        x_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        x_test, y_test = X.iloc[test_idx], y.iloc[test_idx]

        alg = model(**param)
        alg.fit(x_train, y_train,
                eval_set=[(x_test, y_test)],
                early_stopping_rounds=100,verbose = 0)

        pred = alg.predict(x_test)
        mse = np.sqrt(mean_squared_error((y_test), pred))
        print(f"RMSE: {mse}")
    
    return alg,mse

In [ ]:
paramcat = {'learning_rate':0.01,'iterations':4000}
paramlgb = {'learning_rate':0.1,'n_estimators':250}

In [ ]:
crosslgb,mse = cross_val(x,y,LGBMRegressor,paramlgb)
rmse_score['KFold Lightgbm'] = mse


In [ ]:
crosscat,mse = cross_val(x,y,CatBoostRegressor,paramcat)
rmse_score['KFold CatBosst'] = mse

In [ ]:
key = list(rmse_score.keys())

In [ ]:
vals = [float(rmse_score[k]) for k in key]

<h1 id="heading7">
<span style="font-size:30px;color:#800080"><center>Score comparison of different models.</center></span>
<a class="anchor-link" href="https://www.kaggle.com/alampalsingh/notebook004d896e8c#heading7" target ='_self'></a>
</h1>

In [ ]:
col = sns.color_palette('coolwarm',8)

In [ ]:
 col = col.as_hex()

In [ ]:
plt.figure(figsize = (16,6))
score = sns.barplot(vals,key,palette="coolwarm_r")
for i in range(0,len(key)):
    score.text(vals[i]+0.004,i,str(np.round(vals[i],4)),fontdict = dict(color = col[i],fontsize = 12,ha = 'center',va = 'center'),weight = 'bold')

<center><span style="font-size:30px;color:#800080;font-weight:bold">So, CatBoost with KFold is our Winner!!🎉🎉🎉

In [ ]:
pred = np.exp(crosscat.predict(test))
pred = pd.DataFrame(pred)
sub = pd.concat([iD,pred],axis =1)
sub.columns = ['Id','SalePrice']
sub.to_csv('submission.csv',index=False)

<span style="font-size:20px;color:#800080">**If you have gotten this far, you might have done**  </span><span style="font-size:20px;color:#66FF00">***upvote***.</span>